# Quantification with pyCROQUET

***

## pyCROQUET input

Dual guide quantification with pyCROQUET requires two types of input data:

1. **guide library** (see [here](https://github.com/cancerit/pycroquet/wiki/Guide-library-format) for format guidance)   
2. **reads** (SAM/BAM/CRAM)

### Preparing the guide library

For the purpose of this example, we can use the guide annotations in the raw count matrix to generate a [pyCROQUET-formatted](https://github.com/cancerit/pycroquet/wiki/Guide-library-format) library.

The raw counts from Gonatopoulos-Pournatzis *et al.* can be found in GEO series [GSE144281](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE144281) and contain guide-level annotations which we can use to build the library file.

To download the raw count matrix:

In [ ]:
wget -P metadata https://ftp.ncbi.nlm.nih.gov/geo/series/GSE144nnn/GSE144281/suppl/GSE144281_paralogLibrary_rawCounts_NovaSeq_18Sept18.txt.gz
gunzip metadata/GSE144281_paralogLibrary_rawCounts_NovaSeq_18Sept18.txt.gz

Now we have the count matrix, let's take a look at first couple of lines (including the header):

In [ ]:
head -2 metadata/GSE144281_paralogLibrary_rawCounts_NovaSeq_18Sept18.txt

The pyCROQUET guide library format can be found [here](https://github.com/cancerit/pycroquet/wiki/Guide-library-format). The table below shows the mapping of count annotations with pyCROQUET fields.

| pyCROQUET field | count annotation |
| --- | --- |
| id | ID |
| sgrna_ids | Cas9.Target.Site \| Cpf1.Target.Site |
| sgrna_seqs | Cas9.Guide \| Cpf1.Guide |
| gene_pair_id | Gene.symbol1 \| Gene.symbol2 |

To build the pyCROQUET library from the count matrix annotations:

In [ ]:
awk -F"\t" -v OFS="\t" '
BEGIN{
    print "##library-type: dual\n#id\tsgrna_ids\tsgrna_seqs\tgene_pair_id"
} NR > 1 {
    print $1, $5"|"$9, $7"|"$10, $3"|"$4
}' metadata/GSE144281_paralogLibrary_rawCounts_NovaSeq_18Sept18.txt > metadata/GSE144281_paralogLibrary.pyCROQUET.tsv

Header lines are prefixed with `##` while the column headings are prefixed with just a single `#`. This is the minimal header and columns required for a pyCROQUET library, determining the library type (`dual`) and the column headings. Individual annotations for paired guides are separated by `|`.

In [ ]:
head -3 metadata/GSE144281_paralogLibrary.pyCROQUET.tsv 

### Downloading FASTQ from SRA

There are 23 samples for which we need the FASTQ files. For this, we used [SRA toolkit](https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/2.11.2/sratoolkit.2.11.2-ubuntu64.tar.gz) version 2.11.2 following the instructions [here](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit) for installation (see also [install_software_dependencies.ipynb](../install_software_dependencies.ipynb)).

In [ ]:
export PATH=$PATH:${PWD}/../bin/sratoolkit.2.11.2-ubuntu64/bin

First, we get the SRA run accessions for the 23 sample paired end FASTQs we want to download.

In [ ]:
run_ids=($(awk -F"\t" 'NR > 1 && NR < 25 {print $2}' metadata/PRJNA603290_GSE144281_sample_metadata.tsv))
printf "%s\n" "${run_ids[@]}"

We use `fastq-dump` to download processed FASTQ from the SRA with the `--split-files` parameter to split the paired end reads into two separate FASTQ files and compress them (`--gzip`):

```
for rid in "${run_ids[@]}"
do
    fastq-dump --split-files --gzip --outdir data "${rid}"
done
```

Looping over the SRA identifiers, we used LSF to submit these as jobs, but alternatively you can just run the command `fastq-dump --split-files --gzip --outdir data "${rid}"` without the `bsub` code to get the files (as above).

In [ ]:
fastq-dump --version

In [ ]:
for rid in "${run_ids[@]}"
do
    bsub -e "logs/${rid}.fastq_dump.e" -o "logs/${rid}.fastq_dump.o" -q normal -M 3000 -R'select[mem>3000] rusage[mem=3000] span[hosts=1]' fastq-dump --split-files --gzip --outdir data "${rid}"
done

We use [samtools](http://www.htslib.org/) version 1.15 (see [install_software_dependencies.ipynb](../install_software_dependencies.ipynb)) to convert paired-end FASTQ to BAM (with `-1` and `-2` as R2 and R1 respectively - i.e. swapped). This is because `dual-guide` mode accepts only SAM, BAM or CRAM.

In [ ]:
export PATH=$PATH:${PWD}/../bin/samtools-1.15/bin

In [ ]:
samtools version

In [ ]:
for rid in "${run_ids[@]}"
do
    echo "Converting ${rid}..."
    bsub -e "logs/${rid}.samtools.e" -o "logs/${rid}.samtools.o" -q normal -M 3000 -R'select[mem>3000] rusage[mem=3000] span[hosts=1]' samtools import -N -1 "data/${rid}_2.fastq.gz" -2 "data/${rid}_1.fastq.gz" -O BAM -o "data/${rid}.bam"
done

***

## Running pyCROQUET

Sample BAMs were quantified against the [guide library](metadata/GSE144281_paralogLibrary.pyCROQUET.tsv) using [pyCROQUET](https://github.com/cancerit/pycroquet) version 1.5.0 (see [install_software_dependencies.ipynb](../install_software_dependencies.ipynb)) using the `dual-guide` mode, `TinQ` ( target in query) boundary mode, 50000 `chunks` with `exact` (no rules) matching, allowing 1 mismatch (`--rules M`), allowing 2 mismatches (`--rules MM`) or allowing 3 mismatches (`--rules MMM`).

In [ ]:
source ../bin/pycroquet_1.5.1/bin/activate
pycroquet --version

In [ ]:
for rid in "${run_ids[@]}"
do
    bsub -e "logs/${rid}.pycroquet.exact.e" -o "logs/${rid}.pycroquet.exact.o" -q long -J "${rid}.pycroquet.exact" -M 36000 -R'select[mem>36000] rusage[mem=36000] span[hosts=1]' -n 6 pycroquet dual-guide -g metadata/GSE144281_paralogLibrary.pyCROQUET.tsv -q "data/${rid}.bam" -s "${rid}" -o "results/${rid}_exact" -c 6 -b TinQ --chunks 50000 -w "tmp/${rid}_exact"
    bsub -e "logs/${rid}.pycroquet.1M.e" -o "logs/${rid}.pycroquet.1M.o" -q basement -J "${rid}.pycroquet.1M" -M 36000 -R'select[mem>36000] rusage[mem=36000] span[hosts=1]' -n 6 pycroquet dual-guide -g metadata/GSE144281_paralogLibrary.pyCROQUET.tsv -q "data/${rid}.bam" -s "${rid}" -o "results/${rid}_1M" -c 6 -b TinQ --chunks 50000 --rules M -w "tmp/${rid}_1M"
    bsub -e "logs/${rid}.pycroquet.2M.e" -o "logs/${rid}.pycroquet.2M.o" -q basement -J "${rid}.pycroquet.2M" -M 36000 -R'select[mem>36000] rusage[mem=36000] span[hosts=1]' -n 6 pycroquet dual-guide -g metadata/GSE144281_paralogLibrary.pyCROQUET.tsv -q "data/${rid}.bam" -s "${rid}" -o "results/${rid}_2M" -c 6 -b TinQ --chunks 50000 --rules MM -w "tmp/${rid}_2M"
    bsub -e "logs/${rid}.pycroquet.3M.e" -o "logs/${rid}.pycroquet.3M.o" -q basement -J "${rid}.pycroquet.3M" -M 36000 -R'select[mem>36000] rusage[mem=36000] span[hosts=1]' -n 6 pycroquet dual-guide -g metadata/GSE144281_paralogLibrary.pyCROQUET.tsv -q "data/${rid}.bam" -s "${rid}" -o "results/${rid}_3M" -c 6 -b TinQ --chunks 50000 --rules MMM -w "tmp/${rid}_3M"
done

In [ ]:
deactivate

***

## pyCROQUET output



Results from pyCROQUET are stored in the `results` directory:

* `*.counts.tsv.gz` - pyCROQUET-formatted guide abundance
* `*.stats.json` - JSON-formatted screen statistics (e.g. number of reads, number of guides, coverage)
* `*.query_class.tsv.gz` - pyCROQUET-formatted unique read sequence abundance
* `*.cram|*.cram.crai` - CRAM alignment of reads to guides

For more information, please see the [pyCROQUET  Wiki](https://github.com/cancerit/pycroquet/wiki). Intermediate files are stored in `tmp` (the path for these can be set using the `-w` parameter).

*Note: log files (`*.o` or `*.e`) were written in `logs` and are generated by LSF and not pyCROQUET. Log files are not included in this repository.*

We compare the expected counts to the expected_counts (i.e. the published raw counts) to our observed counts (generated by pyCROQUET) in [2_compare_expected_and_observed_counts.ipynb](2_compare_expected_and_observed_counts.ipynb).